## Import the dataset
[coursework doc](https://warwick.ac.uk/fac/sci/dcs/teaching/material/cs342/coursework_cs342_2020-2021.pdf)
[iterative process](https://warwick.ac.uk/fac/sci/dcs/teaching/material/cs342/iterativeprocess.pdf)

Dataset information:
1) sepal length in cm 
2) sepal width in cm
3) petal length in cm
4) petal width in cm

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#import the flower data
df = pd.read_csv("./iris.data", header=None)
print(df)

       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7  3.0  5.2  2.3  Iris-virginica
146  6.3  2.5  5.0  1.9  Iris-virginica
147  6.5  3.0  5.2  2.0  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]


### Convert the dataset labels into numbers rather than text
0 = Iris-setosa
1 = Iris-versicolor
2 = Iris-virginica

In [4]:
df[4].replace('Iris-setosa','0',inplace=True)
df[4].replace('Iris-versicolor','1',inplace=True)
df[4].replace('Iris-virginica','2',inplace=True)

## Method A
Convert the 4D data into 2D using PCA and visualise, based on SVD from lab 5

In [120]:
from sklearn import preprocessing

# Code from lab 5 for PCA using SVD, uses preprocessing from sklearn to centre the data
def pca_decomposition(dataset):
    data_centred = (dataset - dataset.mean()) / dataset.std()
#     data_scaled = preprocessing.scale(dataset)
    U, s, V = np.linalg.svd(data_centred, full_matrices=True)
    return U, s, V

x_features = df[df.columns[:-1]]
x_label = df[df.columns[-1]]
U, s, V = pca_decomposition(x_features)


projected_U = U[:, :2] #only take the first 2 components because we want 2D reduction
projected_U *= s[:2] #you do X * V to project data which is equivalent to U * S so we can just do that

dimension_reduced_iris = pd.DataFrame(data = projected_U, columns = ['PC1', 'PC2'])
#add the labels back to the data so it can be used in the graph..
dimension_reduced_iris["label"] = x_label

plt.figure(figsize=(5, 4))
#setosa
setosa = dimension_reduced_iris.loc[dimension_reduced_iris['label'] == "Iris-setosa"]
plt.scatter(setosa["PC1"], setosa["PC2"], c="r", marker="o")

#versicolor
versicolor = dimension_reduced_iris.loc[dimension_reduced_iris['label'] == "Iris-versicolor"]
plt.scatter(versicolor["PC1"], versicolor[1], c="b", marker="o")

#virginica
virginica = dimension_reduced_iris.loc[dimension_reduced_iris['label'] == "Iris-virginica"]
plt.scatter(virginica["PC1"], virginica["PC2"], c="g", marker="o")

plt.xlabel("PC1")
plt.ylabel("PC2")

plt.show()


         PC1       PC2        label
0  -2.256981 -0.504015  Iris-setosa
1  -2.079459  0.653216  Iris-setosa
2  -2.360044  0.317414  Iris-setosa
3  -2.296504  0.573447  Iris-setosa
4  -2.380802 -0.672514  Iris-setosa
5  -2.063623 -1.513478  Iris-setosa
6  -2.437545 -0.074314  Iris-setosa
7  -2.226383 -0.246787  Iris-setosa
8  -2.334138  1.091490  Iris-setosa
9  -2.181368  0.447131  Iris-setosa
10 -2.156263 -1.067021  Iris-setosa
11 -2.319607 -0.158058  Iris-setosa
12 -2.216657  0.706750  Iris-setosa
13 -2.630902  0.935149  Iris-setosa
14 -2.184972 -1.883668  Iris-setosa
15 -2.243948 -2.713281  Iris-setosa
16 -2.195396 -1.508696  Iris-setosa
17 -2.182866 -0.512587  Iris-setosa
18 -1.887750 -1.426332  Iris-setosa
19 -2.332136 -1.154167  Iris-setosa
20 -1.908164 -0.429028  Iris-setosa
21 -2.197284 -0.949277  Iris-setosa
22 -2.764907 -0.487883  Iris-setosa
23 -1.814333 -0.106394  Iris-setosa
24 -2.220778 -0.161645  Iris-setosa
25 -1.950490  0.605863  Iris-setosa
26 -2.045212 -0.265126  Iris

KeyError: '0'

<Figure size 360x288 with 0 Axes>